# Converting multiple TXT's straight off the acquisition computer to individual HDF5's

This notebook will convert a whole folder of files each to their respective HDF5's, incorporating metadata. It works off the existing file structure created by labview saving.

In [1]:
import phconvert as phc
import csv
phc.__version__
import numpy as np
import os

Point this at a folder which contains folders, which contain data and metadata files together. It will find the data files and metadata files in each folder and create a single HDF5 for each, named according the name you gave the data file (it will automatically chop off the annoying numbers at the beginning and end).

In [3]:
parentfolder = "definitiveset/" #THE DASH AT THE END IS IMPORTANT

# 1.  Read in data

In [4]:
for folder in next(os.walk(parentfolder))[1]:
    for file in os.listdir(parentfolder+folder):        
        if file.endswith("T.txt"):
            datafile = file
            print("converting ", file)
        if file.endswith("etadata.txt"):
            metafile = (os.path.join(parentfolder+folder, file))
        else:
            continue
        savename = datafile[:-8] #these two lines truncate the filename to remove the numbers
        savename = savename[4:]
        savename = parentfolder+savename + ".hdf5"
        datafilefull = (parentfolder+folder+"/"+datafile)
        with open(datafilefull) as inf:                    #everything from here onwards is just normal conversion
            reader = csv.reader(inf, delimiter="	")
            timestamps = list(zip(*reader))[0]    
        with open(datafilefull) as inf:
            reader = csv.reader(inf, delimiter="	")
            detectors = list(zip(*reader))[1]
        detectors = np.asarray(detectors)
        timestamps = np.asarray(timestamps)
        timestamps = np.int64(timestamps)
        detectors = np.uint8(detectors)
        with open(metafile) as inf:
            reader = csv.reader(inf, delimiter=":")
            metastuff = list(zip(*reader))
        User = metastuff[1][3]
        Acquitime = metastuff[1][0]
        Dpower = metastuff[1][1]
        Apower = metastuff[1][2]
        Fluorconc = metastuff[1][4]
        Sampdetails = metastuff[1][5]
        Donor = metastuff[1][6]
        Acceptor = metastuff[1][7]
        buffer = metastuff[1][8]
        Notes = metastuff[1][9]
        DYES = Donor + "," + Acceptor
        description = Notes
        author = User
        author_affiliation = "University of Sheffield"
        sample_name = Sampdetails
        buffer_name = buffer
        dye_names = DYES
        timestamps_unit = 10e-9  # 10 ns, units are always S.I.
        photon_data = dict(
            timestamps=timestamps,
            detectors=detectors,
            timestamps_specs={'timestamps_unit': timestamps_unit})
        setup = dict(
            num_pixels = 2,                   # using 2 detectors
            num_spots = 1,                    # a single confoca excitation
            num_spectral_ch = 2,              # donor and acceptor detection 
            num_polarization_ch = 1,          # no polarization selection 
            num_split_ch = 1,                 # no beam splitter
            modulated_excitation = False,     # CW excitation, no modulation 
            excitation_alternated = [True],  # CW excitation, no modulation 
            lifetime = False,                 # no TCSPC in detection    
            excitation_wavelengths = [515e-9, 638e-9],         # List of excitation wavelenghts
            excitation_cw = [True],                    # List of booleans, True if wavelength is CW
            detection_wavelengths = [580e-9, 690e-9],  # Nominal center wavelength 
        )
        provenance = dict(
            filename=datafilefull, 
            software='Custom Labview Software - Craggs Lab')
        identity = dict(
            author=author,
            author_affiliation=author_affiliation)
        measurement_specs = dict(
            measurement_type = 'smFRET-usALEX',
            detectors_specs = {'spectral_ch1': [0],  # list of donor's detector IDs
                               'spectral_ch2': [1]},  # list of acceptor's detector IDs,
            alex_period = 10000,
            )
        photon_data['measurement_specs'] = measurement_specs
        data = dict(
            description=description,
            photon_data = photon_data,
            setup=setup,
            identity=identity,
            provenance=provenance
        )
        phc.hdf5.save_photon_hdf5(data, h5_fname=savename, overwrite=True)
        print("Saved")
print("All done")

converting  001_1A0_TXT.txt
Saving: definitiveset/1A0.hdf5
Saved
converting  001_1A1_TXT.txt
Saving: definitiveset/1A1.hdf5
Saved
converting  001_1A2_TXT.txt
Saving: definitiveset/1A2.hdf5
Saved
converting  001_1B1_TXT.txt
Saving: definitiveset/1B1.hdf5
Saved
converting  001_1B2_TXT.txt
Saving: definitiveset/1B2.hdf5
Saved
converting  001_1C1_TXT.txt
Saving: definitiveset/1C1.hdf5
Saved
converting  001_1C2_TXT.txt
Saving: definitiveset/1C2.hdf5
Saved
converting  001_6A1_TXT.txt
Saving: definitiveset/6A1.hdf5
Saved
All done
